In [1]:
import pandas as pd

import pyspark
from pyspark.sql.functions import * 

from env import conn

In [2]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

22/10/22 14:10:59 WARN Utils: Your hostname, Morgans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.202 instead (on interface en0)
22/10/22 14:10:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/22 14:11:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/22 14:11:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


1. Read the case, department, and source data into their own spark dataframes.

In [3]:
url = conn('311_data')
q1 = 'SELECT * FROM cases'
q2 = 'SELECT * FROM dept'
q3 = 'SELECT * FROM source'

In [4]:
cases = pd.read_sql(q1, url)

In [5]:
dept = pd.read_sql(q2, url)

In [6]:
source = pd.read_sql(q3, url)

In [7]:
cases = spark.createDataFrame(cases)
dept = spark.createDataFrame(dept)
source = spark.createDataFrame(source)

2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

In [8]:
source.write.json('data/sources_json', mode='overwrite')
source.write.csv('data/sources_csv', mode='overwrite', header=True)

- Inspect your folder structure. What do you notice?

In [9]:
# breaks it down to multiple files in a folder

3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [10]:
cases.show(1, vertical=True)

22/10/22 14:13:36 WARN TaskSetManager: Stage 2 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


22/10/22 14:13:40 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 2 (TID 20): Attempting to kill Python Worker
-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [11]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'bigint')]

In [20]:
cases = cases.withColumnRenamed('SLA_due_date', 'case_due_date')

In [21]:
cases = cases.withColumn('case_closed',
             expr('case_closed =="YES"')
             ).withColumn('case_late',
                         expr('case_late == "YES"'))

In [22]:
cases = cases.withColumn('council_district',
             col('council_district').cast('string')
             )

In [23]:
fmt = 'M/d/yy H:m'

cases = (
    cases.withColumn('case_opened_date', to_timestamp('case_opened_date', fmt))
    .withColumn('case_closed_date', to_timestamp('case_closed_date', fmt))
    .withColumn('case_due_date', to_timestamp('case_due_date', fmt))
)

In [24]:
cases = cases.withColumn('request_address',
              trim(lower('request_address')))

In [27]:
cases = cases.withColumn('case_age',
              datediff(current_timestamp(), 'case_opened_date'))
cases = cases.withColumn('days_to_close',
              datediff('case_closed_date', 'case_opened_date'))
cases = cases.withColumn('case_lifetime',
             when(expr('! case_closed'),
                 col('case_age')
                 )
             .otherwise(col('days_to_close'))
             )

1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [28]:
cases.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'case_age',
 'days_to_close',
 'case_lifetime']

In [30]:
(
    cases.select('case_closed', 'case_lifetime')
    .filter('! case_closed')
    .sort(desc("num_days_late"))
    .where("num_days_late != 'nan'")
).show(2)


22/10/24 09:31:35 WARN TaskSetManager: Stage 14 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


+-----------+-------------+
|case_closed|case_lifetime|
+-----------+-------------+
|      false|         2122|
|      false|         2122|
+-----------+-------------+
only showing top 2 rows



22/10/24 09:32:10 ERROR PythonRunner: Python worker exited unexpectedly (crashed)
java.net.SocketException: Connection reset
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:186)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:252)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:292)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:351)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:200)
	at java.base/java.io.DataInputStream.readFully(DataInputStream.java:170)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:758)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:747)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:512)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at 

2. How many Stray Animal cases are there?

In [33]:
cases.select('service_request_type').show(5)

22/10/24 09:32:34 WARN TaskSetManager: Stage 16 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 09:32:38 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 16 (TID 84): Attempting to kill Python Worker
+--------------------+
|service_request_type|
+--------------------+
|        Stray Animal|
|Removal Of Obstru...|
|Removal Of Obstru...|
|Front Or Side Yar...|
|Animal Cruelty(Cr...|
+--------------------+
only showing top 5 rows



In [36]:
(
    cases.filter(expr('service_request_type == "Stray Animal"')).count()
)

22/10/24 09:34:03 WARN TaskSetManager: Stage 21 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


26760

3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [38]:
(
    cases.filter(cases.dept_division == 'Field Operations')
    .filter(cases.service_request_type != 'Officer Standby')
).count()

22/10/24 09:36:46 WARN TaskSetManager: Stage 25 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


113902

4. Convert the council_district column to a string column.

In [41]:
cases.dtypes

[('case_id', 'bigint'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('case_age', 'int'),
 ('days_to_close', 'int'),
 ('case_lifetime', 'int')]

5. Extract the year from the case_closed_date column.

In [43]:
cases = cases.withColumn('case_closed_year',
                year('case_closed_date'))#.select('case_closed_date', 'case_closed_year').show(5)

6. Convert num_days_late from days to hours in new columns num_hours_late.

In [44]:
cases = cases.withColumn('num_hours_late',
                expr('num_days_late * 24'))

In [45]:
cases.select('num_hours_late', 'num_days_late').show(5)

22/10/24 09:39:47 WARN TaskSetManager: Stage 29 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


22/10/24 09:39:51 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 29 (TID 130): Attempting to kill Python Worker
+-------------------+-------------+
|     num_hours_late|num_days_late|
+-------------------+-------------+
|-23964.210278399998| -998.5087616|
|      -48.302500008| -2.012604167|
|      -72.536111112| -3.022337963|
|      -360.27555552| -15.01148148|
|        8.931944448|  0.372164352|
+-------------------+-------------+
only showing top 5 rows



7. Join the case data with the source and department data.

In [46]:
cases.columns

['case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'dept_division',
 'service_request_type',
 'SLA_days',
 'case_status',
 'source_id',
 'request_address',
 'council_district',
 'case_age',
 'days_to_close',
 'case_lifetime',
 'case_closed_year',
 'num_hours_late']

In [48]:
source.columns

['index', 'source_id', 'source_username']

In [49]:
dept.columns

['dept_division', 'dept_name', 'standardized_dept_name', 'dept_subject_to_SLA']

In [50]:
df = cases.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

In [51]:
df.columns

['source_id',
 'dept_division',
 'case_id',
 'case_opened_date',
 'case_closed_date',
 'case_due_date',
 'case_late',
 'num_days_late',
 'case_closed',
 'service_request_type',
 'SLA_days',
 'case_status',
 'request_address',
 'council_district',
 'case_age',
 'days_to_close',
 'case_lifetime',
 'case_closed_year',
 'num_hours_late',
 'dept_name',
 'standardized_dept_name',
 'dept_subject_to_SLA',
 'index',
 'source_username']

8. Are there any cases that do not have a request source?

In [52]:
df.filter(expr('index is null')).count()

22/10/24 09:49:28 WARN TaskSetManager: Stage 30 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


0

In [54]:
# none. All have a request source

9. What are the top 10 service request types in terms of number of requests?

In [57]:
(
    df.groupBy('service_request_type').count()
    .sort(desc('count'))
).show(10, truncate=False)

22/10/24 09:51:53 WARN TaskSetManager: Stage 66 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



10. What are the top 10 service request types in terms of average days late?

In [62]:
(
    df.groupby('service_request_type')
    .agg(round(mean('num_days_late'), 2).alias('avg_days_late'))
    .sort(desc('avg_days_late'))
    .na.drop()
).show(10, truncate=False)

22/10/24 09:54:46 WARN TaskSetManager: Stage 102 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


+--------------------------------------+-------------+
|service_request_type                  |avg_days_late|
+--------------------------------------+-------------+
|Zoning: Junk Yards                    |175.96       |
|Labeling for Used Mattress            |162.43       |
|Record Keeping of Used Mattresses     |154.0        |
|Signage Requied for Sale of Used Mattr|151.64       |
|Storage of Used Mattress              |142.11       |
|Zoning: Recycle Yard                  |135.93       |
|Donation Container Enforcement        |131.76       |
|License Requied Used Mattress Sales   |128.8        |
|Traffic Signal Graffiti               |101.8        |
|Complaint                             |72.87        |
+--------------------------------------+-------------+
only showing top 10 rows



11. Does number of days late depend on department?

In [67]:
(
    df.groupby('dept_name')
    .mean('num_days_late')
).show(truncate=False)

22/10/24 10:02:51 WARN TaskSetManager: Stage 145 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------+-------------------+
|dept_name                |avg(num_days_late) |
+-------------------------+-------------------+
|Animal Care Services     |-226.51783940550436|
|null                     |135.92851612479797 |
|Solid Waste Management   |-2.2000575136721845|
|Development Services     |13.433724555869714 |
|Trans & Cap Improvements |-20.6128373540527  |
|Customer Service         |59.737091496300785 |
|Metro Health             |-4.911766979607001 |
|Parks and Recreation     |-5.251521960055142 |
|Code Enforcement Services|-38.70133068329558 |
|City Council             |NaN                |
+-------------------------+-------------------+



12. How do number of days late depend on department and request type?

In [68]:
(
    df.groupby('dept_name', 'service_request_type')
    .mean('num_days_late')
    .sort('dept_name')
).show(truncate=False)

22/10/24 10:03:31 WARN TaskSetManager: Stage 157 contains a task of very large size (15174 KiB). The maximum recommended task size is 1000 KiB.


+-------------------------+----------------------------------------+--------------------+
|dept_name                |service_request_type                    |avg(num_days_late)  |
+-------------------------+----------------------------------------+--------------------+
|null                     |Zoning: Recycle Yard                    |135.92851612479797  |
|Animal Care Services     |Aggressive Animal(Critical)             |16.696368811892242  |
|Animal Care Services     |City Council Animal Request             |-1.5475013634821913 |
|Animal Care Services     |Stray Animal                            |-998.8064665118975  |
|Animal Care Services     |Animal Bite(Critical)                   |0.019096834813110194|
|Animal Care Services     |Officer Standby                         |-0.06756953501177049|
|Animal Care Services     |Animal Permits Request                  |22.197922714476803  |
|Animal Care Services     |Aggressive Animal(Non-Critical)         |2.645033883163259   |
|Animal Ca